__Перемножение матриц размером n * m и m * n__

Должна быть возможность варьирования n и m. Матрицы заданного размера генерируются автоматически. Программа должна корректно работать хотя бы до размера входных данных 2500 * 2500 элементов.

## Глобальные переменные

In [1]:
n = 10_000
m = 10_000

## Импорт библиотек

In [2]:
import time

import numpy as np

import torch

import tensorflow as tf

from numba import jit
from numba import prange
from numba import cuda

## Проверка среды GPU

In [3]:
!nvidia-smi

Tue Dec 19 12:08:33 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
torch.cuda.is_available()
# Output would be True if Pytorch is using GPU otherwise it would be False.

True

In [5]:
tf.test.gpu_device_name()
# Standard output is '/device:GPU:0'

'/device:GPU:0'

## CPU-реализации

### Базовая CPU-реализация

In [6]:
def generate_matrix(n, m):
    return np.random.rand(n, m)

def matrix_multiply(matrix1, matrix2):
    return np.dot(matrix1, matrix2)

In [7]:
start_generation_data = time.time()
matrix_a = generate_matrix(n, m)
matrix_b = generate_matrix(m, n)
end_generation_data = time.time()

start_time_CPU = time.time()
result_matrix = matrix_multiply(matrix_a, matrix_b)
end_time_CPU = time.time()

all_time_generation_base = end_generation_data - start_generation_data
all_time_CPU_base = end_time_CPU - start_time_CPU
all_time_base = all_time_generation_base + all_time_CPU_base

In [8]:
print(f"Общее время выполнения на CPU: {all_time_base:.2f} сек")
print(f"Время генерации данных на CPU: {all_time_generation_base:.2f} сек")
print(f"Время перемножения матриц на CPU: {all_time_CPU_base:.2f} сек")

Общее время выполнения на CPU: 72.61 сек
Время генерации данных на CPU: 5.20 сек
Время перемножения матриц на CPU: 67.41 сек


### CPU-реализация с использованием jit-компилятора

In [9]:
# Декоратор @jit компилирует функцию с использованием JIT-компилятора Numba
@jit(nopython=True)
def generate_matrix(n, m):
    return np.random.rand(n, m)

@jit(nopython=True)
def matrix_multiply(matrix1, matrix2):
    return np.dot(matrix1, matrix2)

In [10]:
start_generation_data = time.time()
matrix_a = generate_matrix(n, m)
matrix_b = generate_matrix(m, n)
end_generation_data = time.time()

start_time_CPU = time.time()
result_matrix = matrix_multiply(matrix_a, matrix_b)
end_time_CPU = time.time()

all_time_generation_CPU = end_generation_data - start_generation_data
all_time_CPU = end_time_CPU - start_time_CPU
all_time = all_time_generation_CPU + all_time_CPU

In [11]:
print(f"Общее время выполнения на CPU (с использованием JIT): {all_time:.2f} сек")
print(f"Время генерации данных на CPU: {all_time_generation_CPU:.2f} сек")
print(f"Время перемножения матриц на CPU: {all_time_CPU:.2f} сек")

Общее время выполнения на CPU (с использованием JIT): 68.54 сек
Время генерации данных на CPU: 2.98 сек
Время перемножения матриц на CPU: 65.57 сек


### CPU-реализация с паралельными вычислениями

In [12]:
# Декоратор @jit компилирует функцию с использованием JIT-компилятора Numba
@jit(nopython=True, parallel=True)
def generate_matrix(n, m):
    return np.random.rand(n, m)

@jit(nopython=True, parallel=True)
def matrix_multiply(matrix1, matrix2):
    return np.dot(matrix1, matrix2)

In [ ]:
start_generation_data = time.time()
matrix_a = generate_matrix(n, m)
matrix_b = generate_matrix(m, n)
end_generation_data = time.time()

start_time_CPU = time.time()
result_matrix = matrix_multiply(matrix_a, matrix_b)
end_time_CPU = time.time()

all_time_generation_CPU = end_generation_data - start_generation_data
all_time_CPU = end_time_CPU - start_time_CPU
all_time = all_time_generation_CPU + all_time_CPU

In [14]:
print(f"Общее время выполнения на CPU (с использованием JIT и prange): {all_time:.2f} сек")
print(f"Время генерации данных на CPU: {all_time_generation_CPU:.2f} сек")
print(f"Время перемножения матриц на CPU: {all_time_CPU:.2f} сек")

Общее время выполнения на CPU (с использованием JIT и prange): 70.60 сек
Время генерации данных на CPU: 2.63 сек
Время перемножения матриц на CPU: 67.97 сек


## GPU-реализация

### Реализация в PyTorch

In [24]:
def matrix_multiply_gpu_PT(matrix1, matrix2):
    return torch.mm(matrix1, matrix2)

In [25]:
device = torch.device("cuda")
start_generation_data = time.time()
matrix_a = torch.rand((n, m), device=device)
matrix_b = torch.rand((m, n), device=device)
end_generation_data = time.time()

start_time_GPU = time.time()
result_matrix_gpu = matrix_multiply_gpu_PT(matrix_a, matrix_b)
end_time_GPU = time.time()

all_time_generation_best = end_generation_data - start_generation_data
all_time_GPU_best = end_time_GPU - start_time_GPU
all_time_best = all_time_generation_base + all_time_GPU_best

In [26]:
print(f"Общее время выполнения на GPU (PyTorch): {all_time_best:.6f} сек")
print(f"Время генерации данных на GPU (PyTorch): {all_time_generation_best:.6f} сек")
print(f"Время перемножения матриц на GPU (PyTorch): {all_time_GPU_best:.6f} сек")

Общее время выполнения на GPU (PyTorch): 5.202486 сек
Время генерации данных на GPU (PyTorch): 0.002743 сек
Время перемножения матриц на GPU (PyTorch): 0.001509 сек


In [28]:
all_time_CPU_base / all_time_GPU_best

44680.977402022756

### Рализация в TensorFlow

In [19]:
def matrix_multiply_gpu_TF(matrix1, matrix2):
    return tf.matmul(matrix1, matrix2)

In [20]:
with tf.device('/device:GPU:0'):
    start_generation_data = time.time()
    matrix_a_tf = tf.random.uniform((n, m))
    matrix_b_tf = tf.random.uniform((m, n))
    end_generation_data = time.time()

    start_time_GPU = time.time()
    result_matrix_gpu_tf = matrix_multiply_gpu_TF(matrix_a_tf, matrix_b_tf)
    end_time_GPU = time.time()

all_time_generation = end_generation_data - start_generation_data
all_time_GPU = end_time_GPU - start_time_GPU
all_time = all_time_generation + all_time_GPU

In [21]:
print(f"Общее время выполнения на GPU (TensorFlow): {all_time:.6f} сек")
print(f"Время генерации данных на GPU (TensorFlow): {all_time_generation:.6f} сек")
print(f"Время перемножения матриц на GPU (TensorFlow): {all_time_GPU:.6f} сек")

Общее время выполнения на GPU (TensorFlow): 0.546443 сек
Время генерации данных на GPU (TensorFlow): 0.518758 сек
Время перемножения матриц на GPU (TensorFlow): 0.027684 сек


### Реализации на Cuda

In [22]:
# Функция для генерации матрицы
def generate_matrix(n, m):
    return np.random.rand(n, m)

# Декоратор @jit компилирует функцию с использованием JIT-компилятора Numba для GPU
@cuda.jit
def matrix_multiply_gpu(matrix1, matrix2, result_matrix):
    # Определение индексов i и j внутри сетки CUDA
    i, j = cuda.grid(2)

    # Проверка, что индексы находятся в пределах размеров результирующей матрицы
    if i < result_matrix.shape[0] and j < result_matrix.shape[1]:
        # Инициализация элемента результирующей матрицы
        result_matrix[i, j] = 0.0

        # Вычисление элемента результирующей матрицы
        for k in range(matrix1.shape[1]):
            result_matrix[i, j] += matrix1[i, k] * matrix2[k, j]

In [30]:
# Генерация данных на GPU
start_generation_data = time.time()
d_matrix_a = cuda.to_device(generate_matrix(n, m))
d_matrix_b = cuda.to_device(generate_matrix(m, n))
end_generation_data = time.time()


# Перемножение матриц на GPU

# Инициализация результирующей матрицы для GPU
d_result_matrix_gpu = cuda.device_array((n, n), dtype=np.float32)

# Определение размеров блока и сетки для запуска на GPU
threadsperblock = (16, 16)
blockspergrid_x = (n + threadsperblock[0] - 1) // threadsperblock[0]
blockspergrid_y = (n + threadsperblock[1] - 1) // threadsperblock[1]
blockspergrid = (blockspergrid_x, blockspergrid_y)

# Запуск функции перемножения матриц на GPU с использованием CUDA
start_time_GPU = time.time()
matrix_multiply_gpu[blockspergrid, threadsperblock](d_matrix_a, d_matrix_b, d_result_matrix_gpu)
end_time_GPU = time.time()
# cuda.synchronize()
end_time = time.time()

# Вычисление времени
all_time_generation = end_generation_data - start_generation_data
all_time_GPU = end_time_GPU - start_time_GPU
all_time = end_time - start_generation_data

print(f"Общее время выполнения: {all_time:.6f} сек")
print(f"Время генерации данных: {all_time_generation:.6f} сек")
print(f"Время перемножения матриц на GPU: {all_time_GPU:.6f} сек")

Общее время выполнения: 4.546141 сек
Время генерации данных: 4.543820 сек
Время перемножения матриц на GPU: 0.000465 сек
